In [14]:
import pandas as pd

In [15]:
file_path = 'resultados_secao_absolutos/ALEXANDRE_RAMAGEM_RODRIGUES.csv'
data = pd.read_csv(file_path, index_col=0)
name = file_path.split(r'/')[1].split('.')[0]

In [16]:
data_filtered = data.loc[data['percentual_votos'] < 1].sort_values('percentual_votos', ascending=False)

In [17]:
import tse

database = 'databases/eleicoes_2024.sql'
analise = tse.TseAnalysis(
    database={
        'path': database,
        'alias': 'geral'
    })

analise.attach_database([])

Bancos carregados:
0) main
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----
2) geral
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [18]:
query = """
    SELECT
        NR_LOCAL_VOTACAO AS local,
        NR_ZONA AS zona,
        NR_SECAO AS secao,
        NR_LATITUDE AS latitude,
        NR_LONGITUDE AS longitude,
        QT_ELEITOR_SECAO as eleitores
    FROM
        geral.locais
    WHERE
        CD_MUNICIPIO = 60011
"""
locais = pd.read_sql(query, analise.conn)
locais.head()

,local,zona,secao,latitude,longitude,eleitores
0,1252,185,136,-22.890469,-43.355049,406
1,1341,14,164,-22.887664,-43.302503,446
2,1120,245,130,-22.891374,-43.596135,392
3,1074,185,37,-22.896886,-43.352532,459
4,1023,219,10,-22.844798,-43.339370,390


In [19]:
data_map = pd.merge(
    data_filtered,
    locais,
    how='inner',
    on=['local', 'zona', 'secao']
)
data_map.head()

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,...,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais,percentual_votos,latitude,longitude,eleitores
0,ALEXANDRE RAMAGEM RODRIGUES,60011,1619,204,312,1,71,16,15,16,...,1,0,34,71,0,105,0.676190,-22.901771,-43.184479,105
1,ALEXANDRE RAMAGEM RODRIGUES,60011,1325,204,115,1,74,10,21,25,...,0,0,39,82,0,121,0.611570,-22.907729,-43.173426,121
2,ALEXANDRE RAMAGEM RODRIGUES,60011,1325,204,116,1,71,20,12,24,...,0,0,51,70,0,121,0.586777,-22.907729,-43.173426,121
3,ALEXANDRE RAMAGEM RODRIGUES,60011,1252,245,207,1,130,56,12,57,...,0,0,101,136,0,237,0.548523,-22.902611,-43.564043,237
4,ALEXANDRE RAMAGEM RODRIGUES,60011,1309,182,64,1,109,41,7,77,...,0,0,109,92,0,201,0.542289,-22.921107,-43.369630,201


In [20]:
import folium
from folium.plugins import HeatMap

In [21]:
map = folium.Map(location=[-22.908333, -43.196388])

In [22]:
heat_data = [[row['latitude'], row['longitude'], row['percentual_votos']] for _,row in data_map.iterrows()]

In [23]:
HeatMap(heat_data).add_to(map)

In [24]:
map

In [25]:
map.save(f'calor_{name}.html')